In [ ]:
import sys
sys.path.insert(0,'content/drive/MyDrive/MotorImagery-master')
import IPython

In [ ]:
!pip install BCI2kReader==0.32.dev0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
'''
Functions for preprocessing raw .mat files
'''

import numpy as np
import scipy.signal as sig
import scipy.io as scio
import pandas as pd


def unravel_matfile(file):
    '''
    Unravels data from .mat files produced by LoadMat.m
    '''
    train = file['train']
    HDRtrain=(file['HDRtrain'])
    val_t= HDRtrain[0,0]
    Event= (val_t['EVENT'])
    typtrain = Event['TYP']
    postrain = Event['POS']
    durtrain = Event['DUR']
    
    test = file['eval']
    HDReval=(file['HDReval'])
    val_e= HDReval[0,0]
    Event= (val_e['EVENT'])
    typtest = Event['TYP']
    postest = Event['POS']
    durtest = Event['DUR']

    return [train, typtrain, postrain, durtrain]


def remove_eog(data):
    '''
    Removes EOG channels (last 3 channels)
    '''
    return data[:, :22]


def replace_nans(data):
    '''
    Each trial is separated by either 100 NaNs or 100 of the negative
    maximum value of the channel
    Replace these with the avg value of the channel
    '''
    for i in range(data.shape[1]):
        this_chan = data[:, i]
        data[:, i] = np.where(this_chan == np.min(this_chan), np.nan, this_chan)
        mask = np.isnan(data[:, i])
        chan_mean = np.nanmean(data[:, i])
        data[mask, i] = chan_mean 
    return data


def to_mv(data):
    '''
    Converts v to mV for numerical stability
    '''
    return data * 1e6


def exponential_standardize(data, alpha=0.001, init_block_size=1000, eps=1e-4):
    '''
    This function applies an exponential moving window standardization to the data
    Set first 1000 mean and variance values to the mean and variance of the first 1000 samples
    Computes exponential moving means and variances to standardize data
    Returns channel-wise exponential smooothing with decay factor alpha
    
    data: (num_samples, num_channels)
    returns: (num_samples, num_channels)
    '''
    
    df = pd.DataFrame(data)
    meaned = df.ewm(alpha=alpha).mean()
    demeaned = df - meaned
    squared = demeaned * demeaned
    square_ewmed = squared.ewm(alpha=alpha).mean()
    standardized = demeaned / np.maximum(eps, np.sqrt(np.array(square_ewmed)))
    standardized = np.array(standardized)
    if init_block_size is not None:
        i_time_axis = 0
        init_mean = np.mean(
            data[0:init_block_size], axis=i_time_axis, keepdims=True
        )
        init_std = np.std(
            data[0:init_block_size], axis=i_time_axis, keepdims=True
        )
        init_block_standardized = (
                                          data[0:init_block_size] - init_mean
                                  ) / np.maximum(eps, init_std)
        standardized[0:init_block_size] = init_block_standardized  
    return standardized


def bandpass_filt(data, lowcut=1.0, highcut=40.0, fs=250.0, order=3, filter_type="filtfilt", padlen=None):
    '''
    Bandpass filter  
    '''
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = sig.butter(order, [low, high], btype='band')

    data = data.T
    if filter_type == "filtfilt":
        filtered_data = sig.filtfilt(b, a, data)
    else:
        filtered_data = sig.lfilter(b, a, data)  
    
    return filtered_data.T


def preprocess(data):
    '''
    Performs all preprocessing steps
    '''
    data = remove_eog(data)
    data = replace_nans(data)
    data = to_mv(data)
    
    data = bandpass_filt(data)
    data = exponential_standardize(data)
    return data

def preprocess2(data):
    '''
    Performs all preprocessing steps
    '''
    data = to_mv(data)
    data = bandpass_filt(data)
    data = exponential_standardize(data)
    return data
  

In [ ]:
import numpy as np
import scipy.signal as sig
import scipy.io as scio
import pandas as pd


def extract_trials(data, labels, starts, continuous=False, classes=[]):
    '''
    Extracts individual trials from continuous data (four class)
    
    data: (num_samples, num_channels)
    labels: list of trial labels
    starts: time sample of trial start
    continuous: if True, extract trials with rest
    classes: list of classes to extract, default is empty array to extract all four
             left_hand -> 769, right_hand -> 770, feet -> 771, tongue -> 772
    returns: X: (num_trials, num_samples, num_channels)
             y: (num_trials)
    ''' 
    X = []
    y = []
    new_x=0
    reject = 1023
    
    trial_labels = [769, 770, 771, 772]
    
    
    dur = 1000 # duration of motor imagery task
    if continuous:
        rest = 500 # duration of rest before and after task
    else:
        rest = 0
    for i in np.arange(labels.shape[0]):
    
        # reject bad trials
        if ((i+1 != labels.shape[0]) and (labels[i+1] == reject)):
            continue
            
        # skip labels that are not labels we want to look at
        if len(classes) > 0:
            if labels[i] not in classes:
                continue
        
        
        label = labels[i]
        
        if (label in trial_labels):    
            new_x = data[int(starts[i]-1-rest):int(starts[i]-1+dur+rest)]
            
            if np.asarray(new_x).shape[0] == (dur + 2*rest):
                X.append(new_x)
                y.append(label - 769)   

            
    return np.asarray(X),np.asarray(y)

In [ ]:
def extract_trials_new(data, labels, starts, continuous=False, classes=[]):
    '''
    Extracts individual trials from continuous data (four class)
    
    data: (num_samples, num_channels)
    labels: list of trial labels
    starts: time sample of trial start
    continuous: if True, extract trials with rest
    classes: list of classes to extract, default is empty array to extract all four
             left_hand -> 769, right_hand -> 770, feet -> 771, tongue -> 772
    returns: X: (num_trials, num_samples, num_channels)
             y: (num_trials)
    ''' 
    X = []
    y = []
    new_x=0
    reject = 1023
    
    trial_labels = [1, 2, 3, 4]
    
    dur = 1000 # duration of motor imagery task
    if continuous:
        rest = 875 # duration of rest before and after task
    else:
        rest = 0
    for i in np.arange(labels.shape[0]):
    
        # reject bad trials
        # if ((i+1 != labels.shape[0]) and (labels[i+1] == reject)):
        #     continue
            
        # skip labels that are not labels we want to look at
        if len(classes) > 0:
            if labels[i] not in classes:
                continue
        
        
        label = labels[i]
        
        if (label in trial_labels):    
            new_x = data[int(starts[i]-1-rest):int(starts[i]-1+dur+rest)]
            
            if np.asarray(new_x).shape[0] == (dur + 2*rest):
                X.append(new_x)
                y.append(label - 1)  
    return np.asarray(X),np.asarray(y)

In [ ]:
"""This is for data collected by our team. Imports .dat files and convert to .npy format"""
from sklearn.model_selection import train_test_split
from BCI2kReader import BCI2kReader as b2k
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm 
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA

fs = 256
num_channels = 32

### Things to change for a run
start = 1
end = 6
window = True
random_state = None # Switch to None for shuffling (default)
window_len = [500]

###

if window == True:
  tot_samples = 750
else:
  tot_samples = 1000


# Look at transpose

for j in range(start,end):
    if j < 10:
      data='/content/drive/MyDrive/MotorImagery-master/Data/CollectedData/ArdoWet/ArdoWetS001R0'+str(j)+'.dat'
    else:
      data='/content/drive/MyDrive/MotorImagery-master/Data/CollectedData/ArdoWet/ArdoWetS001R'+str(j)+'.dat'
    path='/content/drive/MyDrive/MotorImagery-master/Data/CollectedData/ArdoWet'
    #print(data)
    with b2k.BCI2kReader(data) as test:
        n=1 
        test.samplingrate

        data = test.signals
        print("Pre transpose: ",data.shape)
        data = np.transpose(data)
        print("Post transpose: ",data.shape)
        # Post transpose matches shape of public dataset

        states = test.states
        stimulusCode = states['StimulusCode'][0]
        #print(stimulusCode)

        startIndeces = []
        endIndeces = []
        labels = []

        midTrial = False

        for i, val in enumerate(stimulusCode):
          if i == 0:
            continue


          if midTrial:
            if stimulusCode[i] == 0:
              endIndeces.append(i-1)
              midTrial = False
            continue

          if stimulusCode[i-1] == 0 and stimulusCode[i]!=0:
            startIndeces.append([i])
            labels.append([stimulusCode[i]])
            midTrial = True

        
        labels = np.array(labels)
        starts = np.array(startIndeces)

        data = preprocess2(data)
      

        print("Data shape: ",data.shape)
        print("Labels shape: ", labels.shape)
        print("Starts shape: ", starts.shape)
        
        if j == start:
          # Make trials
          X, y = extract_trials_new(data, labels, starts, continuous=False) 
          print("X: ",X.shape)
        else:
          X_, y_ = extract_trials_new(data, labels, starts, continuous=False)
          X = np.concatenate((X,X_))
          y = np.concatenate((y,y_))


# Ensure X is shape (trials, samples, channels)

trials, dim1, dim2 = X.shape



if dim1 == num_channels:
    X = np.reshape(X, (trials, dim2, dim1))


print("X shape: !",X.shape)
# Applies moving window to single trial
# stride of 50% of window length

if window == True:
  for L in window_len: #only our 500 window

      # determines starting sample of each window
      if L == 750:
          starts = [0, 100, 250]
      else:
          starts = [0,250,500]
          print("starts: ", starts)
          
    
      # take window
      for i in starts:
          windowed_X = X[:,i:i+L,:]
          print("winXshape: ",windowed_X.shape)
          print("i: ",i)

          
          # split into train-val-test
          X_trainval, X_test, y_trainval, y_test = train_test_split(windowed_X, y, test_size=0.2, random_state=random_state)
          X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.25, random_state=random_state)
          print(window_len[0])
          if window_len[0]==500:
          # save dataset
            print("Subject: ", str(n), " window: ", str(L), " start: ", str(i))
            print()
            np.save(path+'/500Windowed/'+str(i)+'/X_train_'+str(n)+'.npy', X_train)
            np.save(path+'/500Windowed/'+str(i)+'/y_train_'+str(n)+'.npy', y_train)
            np.save(path+'/500Windowed/'+str(i)+'/X_val_'+str(n)+'.npy', X_val)
            np.save(path+'/500Windowed/'+str(i)+'/y_val_'+str(n)+'.npy', y_val)
            np.save(path+'/500Windowed/'+str(i)+'/X_test_'+str(n)+'.npy', X_test)
            np.save(path+'/500Windowed/'+str(i)+'/y_test_'+str(n)+'.npy', y_test)
          if window_len[0]==750:
          # save dataset
            print("Subject: ", str(n), " window: ", str(L), " start: ", str(i))
            print()
            np.save(path+'/750Windowed/'+str(i)+'/X_train_'+str(n)+'.npy', X_train)
            np.save(path+'/750Windowed/'+str(i)+'/y_train_'+str(n)+'.npy', y_train)
            np.save(path+'/750Windowed/'+str(i)+'/X_val_'+str(n)+'.npy', X_val)
            np.save(path+'/750Windowed/'+str(i)+'/y_val_'+str(n)+'.npy', y_val)
            np.save(path+'/750Windowed/'+str(i)+'/X_test_'+str(n)+'.npy', X_test)
            np.save(path+'/750Windowed/'+str(i)+'/y_test_'+str(n)+'.npy', y_test)
else:
    # split into train-val-test
    X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.25,random_state=random_state)
    X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.25,random_state=random_state)
        

    # save dataset
    print("Subject: ", str(i), " window: None")
    print()
    np.save(path+'/NoWindow/X_train_'+str(n)+'.npy', X_train)
    np.save(path+'/NoWindow/y_train_'+str(n)+'.npy', y_train)
    np.save(path+'/NoWindow/X_val_'+str(n)+'.npy', X_val)
    np.save(path+'/NoWindow/y_val_'+str(n)+'.npy', y_val)
    np.save(path+'/NoWindow/X_test_'+str(n)+'.npy', X_test)
    np.save(path+'/NoWindow/y_test_'+str(n)+'.npy', y_test)

print("success")
print()





In [ ]:
"""This is for data made publicly available online. Imports .mat files and convert to .npy format"""
from sklearn.model_selection import train_test_split 
import matplotlib.pyplot as plt
import os

np.random.seed(0)

window_len = [500]
path = "/content/drive/MyDrive/MotorImagery-master/Data/PublicData"
num_subjects = 9
#subject_id = np.arange(num_subjects)
subject_id = [4]
fs = 250
num_channels = 22
tot_samples = 1000

for n in subject_id:
       
    #Load raw data from .mat files
    matdata = unravel_matfile(scio.loadmat("/content/drive/MyDrive/MotorImagery-master/Data/Mat_Files/subject0"+str(n+1)+'.mat') )
    data = matdata[0]
    labels = matdata[1]
    starts = matdata[2]



    data = preprocess(data)
    print(data.shape)


    #labels=np.array(labels)
    labels=labels[0][0]
    #starts=np.array(starts)
    starts=starts[0][0]

    
    
    # Make trials
    X, y = extract_trials(data, labels, starts, continuous=False) 
    print(X.shape)
    print(y.shape)

    # Ensure X is shape (trials, samples, channels)
    trials, dim1, dim2 = X.shape
    if dim1 == num_channels:
        X = np.reshape(X, (trials, dim2, dim1))
        
    
    # Applies moving window to continuous trial
    # Take only first 50% of window length
    window = True

    if window == True:
      for L in window_len: #only our 500 window

          # determines starting sample of each window
          if L == 750:
              starts = [0, 250]
          else:
              starts = np.arange(0, tot_samples-L+1, int(L))
              print("starts: ", starts)
            
       
          # take window
          for i in starts:
              windowed_X = X[:,i:i+L,:]
              print("winXshape: ",windowed_X.shape)
              print("i: ",i)

            
              # split into train-val-test
              X_trainval, X_test, y_trainval, y_test = train_test_split(windowed_X, y, test_size=0.2)
              X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.25)
            

              # save dataset
              print("Subject: ", str(n), " window: ", str(L), " start: ", str(i))
              print()
              np.save(path+'/500Windowed/'+str(i)+'/X_train_'+str(n)+'.npy', X_train)
              np.save(path+'/500Windowed/'+str(i)+'/y_train_'+str(n)+'.npy', y_train)
              np.save(path+'/500Windowed/'+str(i)+'/X_val_'+str(n)+'.npy', X_val)
              np.save(path+'/500Windowed/'+str(i)+'/y_val_'+str(n)+'.npy', y_val)
              np.save(path+'/500Windowed/'+str(i)+'/X_test_'+str(n)+'.npy', X_test)
              np.save(path+'/500Windowed/'+str(i)+'/y_test_'+str(n)+'.npy', y_test)
    else:
      # split into train-val-test
        X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.2)
        X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.25)
            

        # save dataset
        print("Subject: ", str(n), " window: None")
        print()
        np.save(path+'/NoWindow/X_train_'+str(n)+'.npy', X_train)
        np.save(path+'/NoWindow/y_train_'+str(n)+'.npy', y_train)
        np.save(path+'/NoWindow/X_val_'+str(n)+'.npy', X_val)
        np.save(path+'/NoWindow/y_val_'+str(n)+'.npy', y_val)
        np.save(path+'/NoWindow/X_test_'+str(n)+'.npy', X_test)
        np.save(path+'/NoWindow/y_test_'+str(n)+'.npy', y_test)


    print("success")
    print()  

